# Preparación de Proteínas para Docking y DM
Se realiza la preparación final de las proteínas generadas con Modeller, añadiendo átomos faltantes e hidrógenos a las estructuras.

In [1]:
import pdb2pqr
import pdb4amber
import os
from pathlib import Path
import glob

/home/joel/anaconda3/envs/mds/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Se generan las estructuras preparadas 
### (pdb4amber, reduce, pdb2pqr y propka)
La siguiente función recibe como entrada cada una de las estructuras generadas con Modeller (a alas cuales no se le agregaron hidrógenos) y devuelve:
- **Archivos ID_ENS.pdb**: La estructura de la proteína con **todos los hidrógenos añadidos**, pero **no optimizados** para los residuos ionizables. Esto implica que todos sus residuos de histidina serán **HIS**. Además, **todas las conformaciones de la proteína tendrán el mismo número de átomos**, y en las mismas posiciones, lo que permitirá crear un ensamble con ellas.
- **Archivos ID_PROT.pdb**: Estructura de la proteína con hidrógenos añadidos a un pH = 7, para lo cual se ha usado la herramienta **_reduce_** incluida en AmberTools y **PDB2PQR**. Pueden ser usados como estructura inicial para _docking_ y dinámica molecular.
- **Archivos ID_PQR.pdb y ID_PQR.propka**: Archivos resultantes tras la ejecuación de **PDB2PQR** y **propka**.

In [11]:
pdb_list = glob.glob("../ARCHIVOS/CRISTALES/PROT_CDK2_MODELOS_modeller_NoPrep/*")
pdb_list.sort()
for pdb_file in pdb_list[0:1]:
    print(pdb_file)

../ARCHIVOS/CRISTALES/PROT_CDK2_MODELOS_modeller_NoPrep/1aq1_MODLL.pdb


In [12]:
# Directorio actual de trabajo
cwd = os.getcwd()
main_out_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/"
out_dir_ens = main_out_dir +  "PREP_ALL_ENSAMBLE/" # Hidrógenos agregados sin pka
out_dir_PREP_PROT = main_out_dir + "PREP_PH_7/" # Hidrógenos agregados usando Propka (res ionizables a pH 7)
out_dir_pqr = main_out_dir + "PQR_PKAs/" # Directorio de pqr"

# Lista de archivos pdb para preparar, originalmente modelos generados con Modeller
pdb_list = glob.glob("../ARCHIVOS/CRISTALES/PROT_CDK2_MODELOS_modeller_NoPrep/*")
pdb_list.sort()
# Sufijos para nombrar los archivos resultantes
ens_f_tail = "_ENS"
prep_f_tail = "_PROT"

for pdb_file in pdb_list[0:1]:
    print(pdb_file)
    pdb_id = Path(pdb_file).stem.rsplit('_', 1)[0]
    # ENSAMBLE: Añade átomos faltantes, incluyendo Hs
    # No toma en cuenta los estados ionizables
    ensamble_file = out_dir_ens + pdb_id + ens_f_tail + ".pdb"
    pdb4amber.run(arg_pdbin = pdb_file,
                  arg_add_missing_atoms = True, 
                  arg_pdbout = ensamble_file)
    # PROT PREPARADA:
    # Ejecuta pdb2pqr para obtener los estados de protonación de los residuos ionizables
    pqr_file = out_dir_pqr + pdb_id + "_PQR.pdb"
    os.system("pdb2pqr_cli --ff=amber --with-ph=7.0 --ffout=amber --ph-calc-method=propka --chain "
               + pdb_file + " " 
               + pqr_file )
    # Vuelve a ejecutar pdb4amber para generar las PROT con los estados ionizables usando "--reduce"
    prepared_file = out_dir_PREP_PROT  + pdb_id + prep_f_tail + ".pdb"
    pdb4amber.run(arg_pdbin  = pdb_file,
                  arg_reduce = True,
                  arg_pdbout = prepared_file )
    # Elimina los archivos temporales de pdb4amber
    os.system("rm " + out_dir_ens + "/*" + ens_f_tail + "_*")
    os.system("rm " + out_dir_PREP_PROT + "/*" + prep_f_tail + "_*")

../ARCHIVOS/CRISTALES/PROT_CDK2_MODELOS_modeller_NoPrep/1aq1_MODLL.pdb


In [49]:
# Comprobar número de residuos
from prody import *
pdb_reduce = parsePDB(out_dir_pka  + pdb_id + prep_f_tail + ".pdb")
pdb_ens = parsePDB(out_dir_ens + pdb_id + ens_f_tail + ".pdb")
pdb_pqr = parsePQR(out_dir_pqr + pdb_id + "_PQR.pdb")

# Número de histidinas
len(pdb_ens.select("ca resname HIE"))
len(pdb_reduce.select("ca resname HIE"))

pdb_reduce.select("ca resname HID").getResnums()

@> 4848 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 4848 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 4848 atoms and 1 coordinate sets were parsed in 0.02s.


array([121, 125])